In [1]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('adversarial_qa', 'adversarialQA')

     |████████████████████████████████| 542 kB 5.0 MB/s 
     |████████████████████████████████| 118 kB 10.8 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 76 kB 3.4 MB/s 
     |████████████████████████████████| 243 kB 10.8 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


Downloading:   0%|          | 0.00/2.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.02M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset adversarial_qa downloaded and prepared to /root/.cache/huggingface/datasets/adversarial_qa/adversarialQA/1.0.0/92356be07b087c5c6a543138757828b8d61ca34de8a87807d40bbc0e6c68f04b. Subsequent calls will reuse this data.


In [2]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [3]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 149.9 MB 82 kB/s 
     |████████████████████████████████| 57 kB 2.4 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.272 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


In [4]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset["train"])

,id,title,context,question,answers,metadata
0,574c156e2c41bb814208c53b418ff83a7df89766,Madonna_(entertainer),"Madonna's seventh studio album, Ray of Light, (1998) reflected a change in her image. She collaborated with electronica producer William Orbit and wanted to create a sound that could blend dance music with pop and British rock. American music critic Ann Powers explained that what Madonna searched for with Orbit ""was a kind of a lushness that she wanted for this record. Techno and rave was happening in the 90's and had a lot of different forms. There was very experimental, more hard stuff like Aphex Twin. There was party stuff like Fatboy Slim. That's not what Madonna wanted for this. She wanted something more like a singer-songwriter, really. And William Orbit provided her with that.""",What did William Orbit provide?,"{'text': ['a sound that could blend dance music with pop and British rock'], 'answer_start': [164]}","{'split': 'train', 'model_in_the_loop': 'Combined'}"
1,36ada3cb7fd1f6eda512880ab050a6d697a661af,Pub,"A ""country pub"" by tradition is a rural public house. However, the distinctive culture surrounding country pubs, that of functioning as a social centre for a village and rural community, has been changing over the last thirty or so years. In the past, many rural pubs provided opportunities for country folk to meet and exchange (often local) news, while others—especially those away from village centres—existed for the general purpose, before the advent of motor transport, of serving travellers as coaching inns.",Where do people meet to drink,"{'text': ['country pub'], 'answer_start': [3]}","{'split': 'train', 'model_in_the_loop': 'Combined'}"
2,263ba4c25d43a9cc34c936710f5041f17df2fce9,Idealism,"In philosophy, idealism is the group of philosophies which assert that reality, or reality as we can know it, is fundamentally mental, mentally constructed, or otherwise immaterial. Epistemologically, idealism manifests as a skepticism about the possibility of knowing any mind-independent thing. In a sociological sense, idealism emphasizes how human ideas—especially beliefs and values—shape society. As an ontological doctrine, idealism goes further, asserting that all entities are composed of mind or spirit. Idealism thus rejects physicalist and dualist theories that fail to ascribe priority to the mind.",What does the philosophy say about what is real?,"{'text': ['immaterial'], 'answer_start': [170]}","{'split': 'train', 'model_in_the_loop': 'Combined'}"
3,dae8173403c37258105b283ac37519af9b2fe913,Bird,"Most birds can fly, which distinguishes them from almost all other vertebrate classes. Flight is the primary means of locomotion for most bird species and is used for breeding, feeding, and predator avoidance and escape. Birds have various adaptations for flight, including a lightweight skeleton, two large flight muscles, the pectoralis (which accounts for 15% of the total mass of the bird) and the supracoracoideus, as well as a modified forelimb (wing) that serves as an aerofoil. Wing shape and size generally determine a bird species' type of flight; many birds combine powered, flapping flight with less energy-intensive soaring flight. About 60 extant bird species are flightless, as were many extinct birds. Flightlessness often arises in birds on isolated islands, probably due to limited resources and the absence of land predators. Though flightless, penguins use similar musculature and movements to ""fly"" through the water, as do auks, shearwaters and dippers.","Which of the following is not a use of flight: feeding, breeding, or a lightweight skeleton?","{'text': ['lightweight skeleton'], 'answer_start': [276]}","{'split': 'train', 'model_in_the_loop': 'Combined'}"
4,97c72eb6bc9752ba26eb85f59df75d226beb2bcb,Emotion,"Many different disciplines have produced work on the emotions. Human sciences study the role of emotions in mental processes, disorders, and neural mechanisms. In psychiatry, emotions are examined 

In [6]:
!pip install transformers
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

     |████████████████████████████████| 2.6 MB 5.8 MB/s 
     |████████████████████████████████| 3.3 MB 40.3 MB/s 
     |████████████████████████████████| 895 kB 41.8 MB/s 
     |████████████████████████████████| 636 kB 45.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.15
    Uninstalling huggingface-hub-0.0.15:
      Successfully uninstalled huggingface-hub-0.0.15


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [8]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [9]:
pad_on_right = tokenizer.padding_side == "right"

In [10]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [11]:
features = prepare_train_features(dataset['train'][:5])

In [12]:
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [13]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [14]:
batch_size = 16
args = TrainingArguments(
    f"test-adversarialQA",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [15]:
from transformers import default_data_collator

data_collator = default_data_collator

In [16]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [17]:
trainer.train()

***** Running training *****
  Num examples = 30050
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5637


Epoch,Training Loss,Validation Loss
1,2.816600,2.375226
2,2.221500,2.274767
3,1.812600,2.328753


Saving model checkpoint to test-adversarialQA/checkpoint-500
Configuration saved in test-adversarialQA/checkpoint-500/config.json
Model weights saved in test-adversarialQA/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-adversarialQA/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-adversarialQA/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-adversarialQA/checkpoint-1000
Configuration saved in test-adversarialQA/checkpoint-1000/config.json
Model weights saved in test-adversarialQA/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-adversarialQA/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-adversarialQA/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-adversarialQA/checkpoint-1500
Configuration saved in test-adversarialQA/checkpoint-1500/config.json
Model weights saved in test-adversarialQA/checkpoint-1500/pytorch_model.bin
tokenizer config file saved i

TrainOutput(global_step=5637, training_loss=2.382998599298926, metrics={'train_runtime': 3591.438, 'train_samples_per_second': 25.101, 'train_steps_per_second': 1.57, 'total_flos': 2.35559026197504e+16, 'train_loss': 2.382998599298926, 'epoch': 3.0})

In [ ]:
trainer.save_model("BERT_fineTunned_on_AdversarialQA-trained")

Saving model checkpoint to BERT_fineTunned_on_AdversarialQA-trained
Configuration saved in BERT_fineTunned_on_AdversarialQA-trained/config.json
Model weights saved in BERT_fineTunned_on_AdversarialQA-trained/pytorch_model.bin
tokenizer config file saved in BERT_fineTunned_on_AdversarialQA-trained/tokenizer_config.json
Special tokens file saved in BERT_fineTunned_on_AdversarialQA-trained/special_tokens_map.json
